# 导入环境

In [8]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling,Qwen3ForCausalLM
import torch

# 下载Qwen3-8B模型

In [9]:
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen3-8B', cache_dir='/root/Qwen', revision='master')
print(f"模型下载完成，保存路径为：{model_dir}")

2025-06-03 05:19:24,648 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


模型下载完成，保存路径为：/root/Qwen/Qwen/Qwen3-8B


In [10]:
df = pd.read_json("/root/llm_practice/dataset/huanhuan.json")
ds = Dataset.from_pandas(df)
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [11]:
tokenizer = AutoTokenizer.from_pretrained('/root/Qwen/Qwen/Qwen3-8B')

In [12]:
model = Qwen3ForCausalLM.from_pretrained('/root/Qwen/Qwen/Qwen3-8B', device_map="auto",torch_dtype=torch.bfloat16)
model

TypeError: argument of type 'NoneType' is not iterable

# 处理数据集

In [ ]:
messages = [
    {"role": "system", "content": "===system_message_test==="},
    {"role": "user", "content": "===user_message_test==="},
    {"role": "assistant", "content": "===assistant_message_test==="},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_think=True
)

print(text)   # 用于测试chat模版的样子

In [ ]:
def process_func(example):
    MAX_LENGTH = 1024 # 设置最大序列长度为1024个token
    input_ids, attention_mask, labels = [], [], [] # 初始化返回值
    